In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from mysql.connector import errorcode, IntegrityError, DatabaseError
import time

In [2]:
# Load the datasets
customers = pd.read_csv('C:/portfolio/bank_project/data/customers.csv')
accounts = pd.read_csv('C:/portfolio/bank_project/data/accounts.csv')
transactions = pd.read_csv('C:/portfolio/bank_project/data/transactions.csv')


#### Data Cleaning

In [3]:
# Drop rows with missing customer_id or name
customers.dropna(subset=['customer_id', 'name'], inplace=True)

# Replace outlier age > 100 with median
median_age = customers[customers['age'] < 100]['age'].median()
customers.loc[customers['age'] > 100, 'age'] = median_age

# Fill missing age with median
customers['age'].fillna(median_age, inplace=True)

# Fill missing risk_score with 0.5 (neutral)
customers['risk_score'].fillna(0.5, inplace=True)

In [4]:
# Drop negative balances
accounts = accounts[accounts['balance'] >= 0]

In [5]:
# Drop missing amount
transactions = transactions.dropna(subset=['amount'])

# Cap outliers at 99th percentile
cap = transactions['amount'].quantile(0.99)
transactions.loc[transactions['amount'] > cap, 'amount'] = cap


In [6]:
# Check for duplicate account_ids
duplicates = accounts[accounts.duplicated('account_id')]
print("Duplicate account_ids:\n", duplicates)

Duplicate account_ids:
     account_id customer_id account_type   open_date  balance
641   ACC81730    CUST1480     Checking  2023-03-17   3401.9


In [7]:
# Optionally drop them:
accounts = accounts.drop_duplicates('account_id')

### MySQL Connection and Data Insertion

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="dentalclinic",
    database="bank"
)
cursor = conn.cursor()

def insert_in_chunks(df, insert_sql, table_name, chunk_size=100):
    print(f"Inserting into {table_name}...")
    total = len(df)
    for i in range(0, total, chunk_size):
        chunk = df.iloc[i:i + chunk_size]
        for _, row in chunk.iterrows():
            try:
                cursor.execute(insert_sql, tuple(row))
            except IntegrityError:
                pass  # Duplicate primary key
            except DatabaseError as e:
                print(f"Lock timeout in {table_name}: {e}. Retrying...")
                time.sleep(2)  # wait and retry
                try:
                    cursor.execute(insert_sql, tuple(row))
                except:
                    continue
        conn.commit()
    print(f"{table_name} loaded.\n")

# Insert customers
customer_sql = """
INSERT INTO customers (customer_id, name, age, city, risk_score)
VALUES (%s, %s, %s, %s, %s)
"""
insert_in_chunks(customers, customer_sql, "customers")

# Insert accounts
accounts = accounts.drop_duplicates('account_id')
account_sql = """
INSERT INTO accounts (account_id, customer_id, account_type, open_date, balance)
VALUES (%s, %s, %s, %s, %s)
"""
insert_in_chunks(accounts, account_sql, "accounts")

# Insert transactions
transactions = transactions.drop_duplicates('transaction_id')
tx_sql = """
INSERT INTO transactions (transaction_id, account_id, timestamp, amount, type, is_fraud)
VALUES (%s, %s, %s, %s, %s, %s)
"""
insert_in_chunks(transactions, tx_sql, "transactions")

cursor.close()
conn.close()
print("All data inserted successfully.")


Inserting into customers...
✅ customers loaded.

Inserting into accounts...
✅ accounts loaded.

Inserting into transactions...
✅ transactions loaded.

🎉 All data inserted successfully.
